# 3. 数据规整化：合并、清理、过滤

In [1]:
!cd

C:\Code\python\pandas_notebook


In [2]:
import numpy as np
import pandas as pd

## 3.1 入门

----
### 3.1.1 函数说明

`pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort = False)`

concat函数本质上是在所有索引上同时进行对齐合并，而如果想在任意**列**上对齐合并，则需要merge函数，其在sql应用很多。
- left,right： 两个要对齐合并的DataFrame；
- how： 先做笛卡尔积操作，然后按照要求，保留需要的，缺失的数据填充NaN；
   - left: 以左DataFrame为基准，即左侧DataFrame的数据全部保留（不代表完全一致、可能会存在复制），保持原序
   - right: 以右DataFrame为基准，保持原序
   - inner: 交，保留左右DataFrame在on上完全一致的行，保持左DataFrame顺序
   - outer: 并，按照字典顺序重新排序
- on：列索引或列索引列表，如果要在DataFrame相同的列索引做对齐，用这个参数；用于连接的列名，必须同时存在于左右两个DataFrame对象中，如果未指定，则以left和right列名的交集作为连接键
- left_on, right_on, left_index, right_index：
   - on对应普通的列索引或列索引列表，对齐不同列名的DataFrame，用这俩参数；
   - index对应要使用的index，建议不要使用这俩参数，因为可以用concat方法代替。
   - left_on	左侧DataFarme中用作连接键的列
   - right_on	右侧DataFarme中用作连接键的列
   - left_index	将左侧的行索引用作其连接键
   - right_index	将右侧的行索引用作其连接键
- sort: True or False，是否按字典序重新排序。根据连接键对合并后的数据进行排序，默认为True。有时在处理大数据集时，禁用该选项可获得更好的性能

In [3]:
df1 = pd.DataFrame([[1,2],[3,4]], index = ['a','b'],columns = ['A','B'])
df2 = pd.DataFrame([[1,3],[4,8]], index = ['b','d'],columns = ['B','C'])
df1

,A,B
a,1,2
b,3,4


In [4]:
df2

,B,C
b,1,3
d,4,8


如果单纯的按照index对齐，不如用concat方法，所以一般不建议使用left_index,  right_index  。

In [4]:
pd.merge(left = df1, right = df2, how = 'inner' ,left_index = True, right_index = True)

,A,B_x,B_y,C
b,3,4,1,3


In [5]:
pd.merge(left = df1, right = df2, how = 'outer' ,left_index = True, right_index = True)

,A,B_x,B_y,C
a,1.0,2.0,NaN,NaN
b,3.0,4.0,1.0,3.0
d,NaN,NaN,4.0,8.0


In [6]:
# 小区别是concat对重复列没有重命名，但是重名的情况不多，而且重名了说明之前设计就不大合理。
pd.concat([df1,df2], join = 'inner',axis =1)  

,A,B,B,C
b,3,4,1,3


--- 
### 3.1.2. `on` 用法
设置 `how = 'inner'`

In [7]:
#对于'B'列：df1的'b'行、df2的'd'行，是相同的，其他都不同。 
pd.merge(left = df1, right = df2, how = 'inner' , on =['B']) 

,A,B,C
0,3,4,8


In [8]:
# df1的'A'列'b'行，df2的'C'列'd'行是相同的，其他都不同。
# 其他列如果同名会进行重命名。
pd.merge(left = df1, right = df2, how = 'inner',left_on = ['A'] ,right_on = ['C'])

,A,B_x,B_y,C
0,3,4,1,3


### 3.1.3 `how` 用法

In [9]:
# 保持左侧DataFrame不变，用右侧来跟它对齐，对不上的填NaN。
pd.merge(left = df1, right = df2, how = 'left', on = ['B'] )

,A,B,C
0,1,2,NaN
1,3,4,8.0


In [10]:
# 保持右侧DataFrame不变，用右侧来跟它对齐，对不上的填NaN。
pd.merge(left = df1, right = df2, how = 'right', on = ['B'] )

,A,B,C
0,3.0,4,8
1,NaN,1,3


对齐的列存在重复值

重复的也没关系，操作逻辑是一致的，完全可以假想不存在重复。

In [11]:
df1.loc['a','B'] =4  #改成重复
df1

,A,B
a,1,4
b,3,4


In [12]:
### 保持右侧的列都在，如果左侧对齐的列存在重复值，那么对齐上后也存在重复。
pd.merge(left = df1, right = df2, how = 'right', on = ['B'] )

,A,B,C
0,1.0,4,8
1,3.0,4,8
2,NaN,1,3


## 3.2 多对一的合并

一个表的连接键列有重复值，另一个表中的连接键没有重复值

In [7]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [8]:
df2 = pd.DataFrame({'key':['a','b','d'],'data2':range(3)})

df2

,data2,key
0,0,a
1,1,b
2,2,d


In [9]:
pd.merge(df1,df2)#默认情况

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [10]:
df1.merge(df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [11]:
df1.merge(df2,on = 'key',how = 'inner')#内连接，取交集

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [12]:
df1.merge(df2,on = 'key',how = 'outer')#外链接，取并集，并用nan填充

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [13]:
df1.merge(df2,on = 'key',how = 'left')#左连接，左侧DataFrame取全部，右侧DataFrame取部分

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [14]:
df1.merge(df2,on = 'key',how = 'right')#右连接，右侧DataFrame取全部，左侧DataFrame取部分

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


如果左右侧DataFrame的连接键列名不一致，但是取值有重叠，可使用left_on、right_on来指定左右连接键

In [15]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1': range(7)})

df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [16]:
df4 = pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})

df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [17]:
df3.merge(df4,left_on = 'lkey',right_on = 'rkey',how = 'inner')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


## 3.3 多对多的合并

### 3.3.1 一个表的连接键列有重复值，另一个表中的连接键有重复值

In [3]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1': range(7)})

df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [4]:
df5 = pd.DataFrame({'key':['a','b','a','b','b'],'data2': range(5)})

df5

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,b


In [6]:
df1.merge(df5)

,data1,key,data2
0,0,b,1
1,0,b,3
2,0,b,4
3,1,b,1
4,1,b,3
5,1,b,4
6,6,b,1
7,6,b,3
8,6,b,4
9,2,a,0


df1 3a * df5 2a    +  df1 3b * df5 3b  ==> 15


合并小结

1）默认情况下，会将两个表中相同列名作为连接键

2）多对多，会采用笛卡尔积形式链接（左表连接键有三个值‘1，3，5’，右表有两个值‘2，3’，则会形成，（1，2）（1，3）（3，1），（3，2）。。。6种组合）

在如，假设集合A={a, b}，集合B={0, 1, 2}，则两个集合的笛卡尔积为{(a, 0), (a, 1), (a, 2), (b, 0), (b, 1), (b, 2)}。

### 3.3.2 存在多个连接键的处理

In [7]:
left = pd.DataFrame({'key1':['foo','foo','bar'],'key2':['one','one','two'],'lval':[1,2,3]})

right = pd.DataFrame({'key1':['foo','foo','bar','bar'],'key2':['one','one','one','two'],'rval':[4,5,6,7]})

In [8]:
left

,key1,key2,lval
0,foo,one,1
1,foo,one,2
2,bar,two,3


In [9]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,one,2,4
3,foo,one,2,5
4,bar,two,3,7


1）连接键是多对多关系，应执行笛卡尔积形式

2）多列应看连接键值对是否一致

In [13]:
pd.merge(left,right ,on = ['key1','key2'],how = 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4
1,foo,one,1.0,5
2,foo,one,2.0,4
3,foo,one,2.0,5
4,bar,two,3.0,7
5,bar,one,NaN,6


### 3.3.3 对连接表中非连接列的重复列名的处理

In [14]:
pd.merge(left,right,on = 'key1')  # key2不是连接键，有重复列

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,one,2,one,4
3,foo,one,2,one,5
4,bar,two,3,one,6
5,bar,two,3,two,7


In [15]:
pd.merge(left,right,on = 'key1',suffixes = ('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,one,2,one,4
3,foo,one,2,one,5
4,bar,two,3,one,6
5,bar,two,3,two,7


## 3.4 索引上的合并

当连接键位于索引中时，成为索引上的合并，可以通过merge函数，传入left_index、right_index来说明应该被索引的情况。

### 3.4.1 一表中连接键是索引列、另一表连接键是非索引列

In [17]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [18]:
right1 = pd.DataFrame({'group_val':[3.5,7]},index = ['a','b'])
right1

,group_val
a,3.5
b,7.0


In [19]:
pd.merge(left1,right1,left_on = 'key',right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


有上可知，left_on、right_on是指定表中非索引列为连接键，left_index、right_index是指定表中索引列为连接键，两者可以组合，是为了区分是否是索引列

### 3.4.1 两个表中的索引列都是连接键

In [20]:
left2 = pd.DataFrame(np.arange(6).reshape(3,2),index = ['a','b','e'],columns = ['0hio','nevada'])

right2 = pd.DataFrame(np.arange(7,15).reshape(4,2),index = ['b','c','d','e'],columns = ['misso','ala'])


In [21]:
left2

,0hio,nevada
a,0,1
b,2,3
e,4,5


In [22]:
right2

,misso,ala
b,7,8
c,9,10
d,11,12
e,13,14


In [24]:
pd.merge(left2,right2,left_index = True,right_index = True,how = 'outer')

,0hio,nevada,misso,ala
a,0.0,1.0,NaN,NaN
b,2.0,3.0,7.0,8.0
c,NaN,NaN,9.0,10.0
d,NaN,NaN,11.0,12.0
e,4.0,5.0,13.0,14.0


## 3.5 轴向连接

在这里展示一种新的连接方法，对应于numpy的concatenate函数，pandas有concat函数

In [26]:
arr =np.arange(12).reshape(3,4)

In [27]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [28]:
np.concatenate([arr,arr],axis = 1)#横轴连接块

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

### 3.5.1 concat函数参数表格

    参数	说明

objs	参与连接的列表或字典，且列表或字典里的对象是pandas数据类型，唯一必须给定的参数

axis=0	指明连接的轴向，0是纵轴，1是横轴，默认是0

join ‘inner’（交集），‘outer’（并集），默认是‘outer’指明轴向索引的索引是交集还是并集

join_axis  指明用于其他n-1条轴的索引（层次化索引，某个轴向有多个索引），不执行交并集

keys	与连接对象有关的值，用于形成连接轴向上的层次化索引（外层索引），可以是任意值的列表              或数组、元组数据、数组列表（如果将levels设置成多级数组的话）

levels	指定用作层次化索引各级别（内层索引）上的索引，如果设置keys的话

names 用于创建分层级别的名称，如果设置keys或levels的话

verify_integrity	检查结果对象新轴上的重复情况，如果发横则引发异常，默认False，允许重复

ignore_index	不保留连接轴上的索引，产生一组新索引range（total_length）

In [29]:
s1 = pd.Series([0,1,2],index = ['a','b','c'])

s2 = pd.Series([2,3,4],index = ['c','f','e'])

s3 = pd.Series([4,5,6],index = ['c','f','g'])

In [30]:
pd.concat([s1,s2,s3])#默认并集、纵向连接

a    0
b    1
c    2
c    2
f    3
e    4
c    4
f    5
g    6
dtype: int64

In [31]:
pd.concat([s1,s2,s3],ignore_index = True)#生成纵轴上的并集，索引会自动生成新的一列

0    0
1    1
2    2
3    2
4    3
5    4
6    4
7    5
8    6
dtype: int64

In [33]:
pd.concat([s1,s2,s3],axis = 1,join = 'inner')#横向取交集,注意该方法对对象表中有重复索引时失效

,0,1,2
c,2,2,4


In [34]:
pd.concat([s1,s2,s3],axis = 1,join = 'outer')#横向索引取并集，纵向索引取交集,注意该方法对对象表中有重复索引时失效

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,2.0,2.0,4.0
e,NaN,4.0,NaN
f,NaN,3.0,5.0
g,NaN,NaN,6.0


### 3.5.2 concat函数小结

1）纵向连接，ignore_index = False ,可能生成重复的索引

2）横向连接时，对象索引不能重复

## 3.6 合并重叠数据

适用范围：

1）当两个对象的索引有部分或全部重叠时

2）用参数对象中的数据为调用者对象的缺失数据‘打补丁’

In [38]:
a = pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],index = ['a','b','c','d','e','f'])

b = pd.Series(np.arange(len(a)),index = ['a','b','c','d','e','f'])

In [39]:
a

a    NaN
b    2.5
c    NaN
d    3.5
e    4.5
f    NaN
dtype: float64

In [40]:
b

a    0
b    1
c    2
d    3
e    4
f    5
dtype: int32

In [41]:
a.combine_first(b)#利用b填补了a的空值

a    0.0
b    2.5
c    2.0
d    3.5
e    4.5
f    5.0
dtype: float64

In [42]:
a = pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],index = ['g','b','c','d','e','f'])

In [43]:
a

g    NaN
b    2.5
c    NaN
d    3.5
e    4.5
f    NaN
dtype: float64

In [44]:
a.combine_first(b)#部分索引重叠

a    0.0
b    2.5
c    2.0
d    3.5
e    4.5
f    5.0
g    NaN
dtype: float64

## 3.7 移除重复数据（duplicated）

检查某列数据是否重复可用.is_unique

检查某行数据是否重复可用.duplicated

In [3]:
import pandas as pd
import numpy as np

data = pd.DataFrame({'k1':['one'] * 3 + ['two'] * 4,'k2':[1,1,2,3,3,4,4]})

data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


### 3.7.1 检查列以及行中重复数据

In [4]:
data.index.is_unique#检查索引(列)

True

In [5]:
data.k1.is_unique#检查列

False

In [6]:
data['k2'].is_unique#检查列

False

In [ ]:
#
data.is_unique  #不支持

In [7]:
data.duplicated() #检查行

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

### 3.7.2 移除重复行

In [9]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [10]:
set(data.k1)  #保留唯一的列属性值

{'one', 'two'}

移除重复值小结

1) drop_duplicates、duolicated函数只能用于DataFrame

2) is_unique不能用于DataFrame

## 3.8 利用函数或映射进行数据转换

In [17]:
data1 = pd.DataFrame({'food':['bacon','pork','bacon','Pastrami',\
                              'beef','Bacon','pastrami','ham','lox'],\
                      'ounces':[4,3,12,6,7.5,8,3,5,6]})

data1

,food,ounces
0,bacon,4.0
1,pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,ham,5.0
8,lox,6.0


添加一列表示肉类来源的动物类型

In [18]:
#step1:构建肉类到动物的映射
meat_to_animal = {'bacon':'pig','pork':'pig','pastrami':'cow','beef':'cow','ham':'pig',\
                 'lox':'salmon'}

Series的map方法可以接受一个函数或含有映射关系的字典型对象，字符的大小写要一致

In [19]:
#step2：映射
data1['animal'] = data1['food'].map(str.lower).map(meat_to_animal)

data1

,food,ounces,animal
0,bacon,4.0,pig
1,pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,ham,5.0,pig
8,lox,6.0,salmon


In [20]:
#step2的另一种实现方法

data1['food'].map(lambda x:meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

map是一种实现元素级转换记忆其他数据清理工作的便捷方式

map会改变原始数据集

## 3.9 替换值

替换缺失值的方法：

1）fillna

2）含有重复索引的合并combine_first

3）replace

In [21]:
data2 = pd.Series([1.,-999,2,-999,-1000,3.])

data2

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999可能是一个表示缺失数据的标记值，要将其替换为pandas能够理解的NA值，可以利用replace

In [22]:
data2.replace(-999,np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [23]:
data2  # replace不改变原数据集

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

一次性替换多个值

In [24]:
data2.replace([-999,-1000],np.nan)#一次传入一个列表即可

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [25]:
data2.replace([-999,-1000],[np.nan,0]) #列表对应

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [26]:
data2.replace({-999:np.nan,-1000:0}) #字典

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## 3.10 重命名轴索引

跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新对象，轴还可以被就地修改，而无需新建一个数据结构

In [28]:
data3 = pd.DataFrame(np.arange(12).reshape(3,4),\
                     index = ['a','b','c'],columns = ['one','two','three','four'])

In [31]:
data3

,one,two,three,four
a,0,1,2,3
b,4,5,6,7
c,8,9,10,11


In [30]:
data3.index.map(str.upper)

Index(['A', 'B', 'C'], dtype='object')

In [32]:
data3

,one,two,three,four
a,0,1,2,3
b,4,5,6,7
c,8,9,10,11


In [34]:
data3.index = data3.index.map(str.upper)#修改了

data3

,one,two,three,four
A,0,1,2,3
B,4,5,6,7
C,8,9,10,11


还可以通过rename结合字典型对象实现对部分轴标签的更新

In [35]:
data3.rename(index = {'A':'aaa'},columns = {'three':'liu'})

,one,two,liu,four
aaa,0,1,2,3
B,4,5,6,7
C,8,9,10,11


In [36]:
data3  #不改变原数据

,one,two,three,four
A,0,1,2,3
B,4,5,6,7
C,8,9,10,11


In [37]:
data3 = data3.rename(index = {'A':'aaa'},columns = {'three':'liu'})

data3 #改变

,one,two,liu,four
aaa,0,1,2,3
B,4,5,6,7
C,8,9,10,11


## 3.11 检测和过滤异常值

这里的异常值的阈值已知，因此，异常值的过滤或变换运算很大程度上其实就是逻辑数组运算。

In [38]:
data4 = pd.DataFrame(np.random.randn(1000,4))

In [39]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
0    1000 non-null float64
1    1000 non-null float64
2    1000 non-null float64
3    1000 non-null float64
dtypes: float64(4)
memory usage: 31.3 KB


In [40]:
data4.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.000369,0.021052,0.001597,-0.004234
std,0.996418,0.995939,1.025827,0.975555
min,-2.831989,-3.263754,-2.946051,-3.582173
25%,-0.677185,-0.679675,-0.697295,-0.683853
50%,-0.045373,0.036025,-0.003890,-0.010966
75%,0.708086,0.694228,0.699752,0.678483
max,3.300092,3.118834,3.226208,2.837254


In [42]:
data4.head(5)

,0,1,2,3
0,-1.346436,-0.860207,-0.763197,-0.435268
1,-1.401356,1.147635,-0.360043,1.613449
2,-0.617555,-0.832592,1.271173,1.324569
3,-1.937709,-0.394873,0.195300,-0.694396
4,0.088919,-0.963438,-0.803196,-0.386243


In [41]:
## 找出某列绝对值大于3的值  列3

data4[3][np.abs(data4[3]) > 3]

67    -3.582173
993   -3.038611
995   -3.075145
Name: 3, dtype: float64

In [45]:
# 找出全部或含有“超过3或-3的值”的行

(np.abs(data4) > 3).any(1).head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [46]:
data4[(np.abs(data4) > 3).any(1)]

,0,1,2,3
67,-0.374106,1.977601,2.997721,-3.582173
142,1.225765,0.666180,3.047576,0.931909
271,-0.678715,-0.704692,3.105890,-0.011481
385,3.300092,0.814611,-1.076080,-0.802785
668,-1.364173,3.118834,1.412935,2.033025
872,0.829088,1.131947,3.226208,0.666931
956,0.960789,-3.263754,0.654492,-0.585739
993,-1.837448,-1.190133,1.031639,-3.038611
995,-0.312391,-0.564607,-1.192526,-3.075145


In [ ]:
## 替换异常值

data4[np.abs(data4) > 3] = np.sign(data) * 3  # bug


In [54]:
data4.isnull().sum()#有空值

0    0
1    0
2    0
3    0
dtype: int64

In [53]:
data4 = data4.replace(np.nan,0)

data4.isnull().sum()#无空值

0    0
1    0
2    0
3    0
dtype: int64

In [55]:
data4.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.000369,0.021052,0.001597,-0.004234
std,0.996418,0.995939,1.025827,0.975555
min,-2.831989,-3.263754,-2.946051,-3.582173
25%,-0.677185,-0.679675,-0.697295,-0.683853
50%,-0.045373,0.036025,-0.003890,-0.010966
75%,0.708086,0.694228,0.699752,0.678483
max,3.300092,3.118834,3.226208,2.837254


## 3.12 排列和随机采样

1）numpy.random.permutation函数

2）np.random.randint生成随机数

In [56]:
df = pd.DataFrame(np.arange(5 *4).reshape(5,4))

sampler = np.random.permutation(5)

In [57]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [58]:
sampler

array([0, 1, 3, 2, 4])

In [60]:
df.iloc[sampler]

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15
2,8,9,10,11
4,16,17,18,19


In [61]:
df.take(sampler)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15
2,8,9,10,11
4,16,17,18,19


In [62]:
# 通过np.random.randint得到一组随机整数

sampler1 = np.random.randint(0,len(df),size = 4)

In [63]:
sampler1

array([0, 2, 1, 0])

In [64]:
df1 = df.take(sampler1)

In [65]:
df1

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3


## 3.13 计算指标/哑变量

将分类变量（categorical variable）转换为（哑变量矩阵，dummy matrix）或（指标矩阵，indicator matrix）是常用于统计学习建模或机器学习的转换方式。

即 DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值为1或0）。

pandas中的get_dummies函数可以实现以上功能

In [68]:
df2 = pd.DataFrame({'key':['b','a','b','c','a','b'],'data1':range(6)})

In [69]:
df2

,data1,key
0,0,b
1,1,a
2,2,b
3,3,c
4,4,a
5,5,b


In [70]:
pd.get_dummies(df2.key)

,a,b,c
0,0,1,0
1,1,0,0
2,0,1,0
3,0,0,1
4,1,0,0
5,0,1,0


In [71]:
pd.get_dummies(df2['key'],prefix = 'key')

,key_a,key_b,key_c
0,0,1,0
1,1,0,0
2,0,1,0
3,0,0,1
4,1,0,0
5,0,1,0


In [72]:
## get_dummies矩阵和原数据连接

dummies = pd.get_dummies(df2['key'],prefix = 'key')
pd.concat([df2['data1'],dummies],axis = 1)

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,1,0,0
2,2,0,1,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [73]:
df2[['data1']].join(dummies)#Series没有join

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,1,0,0
2,2,0,1,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [74]:
df2[['data1']]#选出的是DataFrame

,data1
0,0
1,1
2,2
3,3
4,4
5,5


In [75]:
df2['data1']#选出的是Series

0    0
1    1
2    2
3    3
4    4
5    5
Name: data1, dtype: int32